In [2]:
import numpy as np
from scipy.io import loadmat

# 加载.mat文件
data = loadmat('further_cleaned_data.mat')
# 提取 AE_spindle 数据
ae_spindle_data = data['AE_spindle']
vib_spindle_data = data['vib_spindle']
ae_table_data = data['AE_table']
vib_table_data = data['vib_table']
VB_data = data['VB']

# Function to remove one in every three values
# def reduce_values(data_array):
#     return data_array[:, np.delete(np.arange(data_array.shape[1]), np.arange(2, data_array.shape[1], 3))]

# Applying the function to each of the variables
# AE_spindle = reduce_values(ae_spindle_data)
# AE_table = reduce_values(ae_table_data)
# vib_spindle = reduce_values(vib_spindle_data)
# vib_table = reduce_values(vib_table_data)
AE_spindle = ae_spindle_data
AE_table = ae_table_data
vib_spindle = vib_spindle_data
vib_table = vib_table_data

# ae_spindle_reduced = AE_spindle[:, start_index_ae:end_index_ae]
# vib_spindle_reduced = vib_spindle[:, start_index_vb:end_index_vb]
# ae_table_reduced = AE_table[:, start_index_ae:end_index_ae]
# vib_table_reduced = vib_table[:, start_index_vb:end_index_vb]


combined_feature = np.stack((AE_spindle,vib_table,vib_spindle), axis=-1)



print(f'Reshaped data shape: {combined_feature.shape},{VB_data.shape}')

Reshaped data shape: (99, 9000, 3),(99, 1)


In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from keras.callbacks import LearningRateScheduler



# Extract features and targets
X = combined_feature #feature
y = VB_data #target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - ss_res/(ss_tot + K.epsilon()))
    
# 初始化模型
model = Sequential()
model.add(LSTM(200, activation='tanh', input_shape=(9000, 3)))
model.add(Dense(1))

optimizer = Adam(learning_rate=0.0001, decay=1e-6,clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mse',metrics=[r_squared])

#学习率衰减
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))
lr_scheduler = LearningRateScheduler(scheduler,verbose = 1)

#早停
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 训练模型
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1,callbacks=[lr_scheduler])

# 模型评估
loss= model.evaluate(X_test, y_test)

print(f'Test Loss: {loss}')



Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/100


C:\Users\17331\.conda\envs\MachineLearn\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\17331\.conda\envs\MachineLearn\lib\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 75s 19s/step - loss: 0.0983 - r_squared: -3.7470 - val_loss: 0.1171 - val_r_squared: -2.3794 - learning_rate: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 73s 19s/step - loss: 0.0748 - r_squared: -2.3377 - val_loss: 0.0963 - val_r_squared: -1.7777 - learning_rate: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 74s 19s/step - loss: 0.0612 - r_squared: -1.6800 - val_loss: 0.0790 - val_r_squared: -1.2794 - learning_rate: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 75s 19s/step - loss: 0.0481 - r_squared: -1.2230 - val_loss: 0.0656 - val_r_squared: -0.8935 - learning_rate: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 74s 19s/step - loss: 0.

In [1]:
import matplotlib.pyplot as plt

# 绘制训练和验证的损失曲线
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()
plt.plot(history.history['r_squared'], label='Train R^2')
plt.plot(history.history['val_r_squared'], label='Validation R^2')
plt.title('R^2')
plt.ylabel('R^2')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()


NameError: name 'history' is not defined

In [9]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model



test_data = loadmat('row_110_data.mat')
ae_spindle_data = test_data['AE_spindle']
vib_spindle_data = test_data['vib_spindle']
ae_table_data = test_data['AE_table']
vib_table_data = test_data['vib_table']
VB_data = test_data['VB']



# ae_spindle_reduced = ae_spindle_data[:, start_index_ae:end_index_ae]
# vib_spindle_reduced = vib_spindle_data[:, start_index_vb:end_index_vb]
# ae_table_reduced = ae_table_data[:, start_index_ae:end_index_ae]
# vib_table_reduced = vib_table_data[:, start_index_ae:end_index_ae]


combined_feature = np.stack((ae_spindle_data,vib_spindle_data,vib_table_data), axis=-1)


# 使用模型进行预测
predictions = model.predict(combined_feature)

# 选择一个样本来展示预测和实际值
sample_index = 0
predicted_values = predictions[sample_index]
actual_values = VB_data[sample_index]
print(predicted_values, actual_values)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
[0.3205801] [0.24]
